In [1]:
import higra as hg 
import numpy as np 
import numpy.ma as ma 
import idefix
from idefix.utils import * 
from idefix.vxl import get_grid, bin, fit_bbox
from idefix.io import load_las 
from sap.utils import *
from custom_maxtree import * 

/home/gentleprotector/miniconda3/envs/letg_internship/lib/python3.12/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/gentleprotector/miniconda3/envs/letg_internship/lib/python3.12/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/gentleprotector/miniconda3/envs/letg_internship/lib/python3.12/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/gentleprotector/miniconda3/envs/letg_internship/lib/python3.12/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(sel

In [2]:
from mayavi import mlab 
mlab.init_notebook() 

QStandardPaths: wrong permissions on runtime directory /run/user/1000/, 0755 instead of 0700


Notebook initialized with ipy backend.


In [71]:
las = load_las('../pc_data/ts3_clip.las')

In [72]:
# Hard outlier filter here
las = las[(las.spatial[..., -1] > 4) & (las.spatial[..., -1] < 25)]

In [73]:
z_min = np.min(las.spatial[..., -1])
z_max = np.max(las.spatial[..., -1])
h_exp = z_max - z_min 
print(f"The expected maximum tree height is {h_exp:.2f}m.".format(h_exp))

The expected maximum tree height is 9.95m.


**Voxelization** Using mean of intensities.

In [88]:
grid_space = 1
bbox = fit_bbox(las.spatial) 
grid = get_grid(bbox, grid_space) 
fval = getattr(las.feature, 'intensity')
vxl = bin(grid, las.spatial, fval, 'mean') 

In [89]:
#vxl

**MaxTree creation**

In [90]:
t = MaxTree(vxl, adjacency=6)

In [91]:
print(t)

MaxTree{'adjacency': 6,
 'image.dtype': dtype('uint16'),
 'image.shape': (44, 59, 10),
 'image_hash': '13642f58',
 'image_name': None,
 'num_nodes': 29619}


In [92]:
mlab.clf()
idefix.vxl.plot(vxl)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [140]:
mlab.clf()

In [94]:
area = t.get_attribute('area')

In [116]:
area

array([1.000e+00, 1.000e+00, 1.000e+00, ..., 4.711e+03, 2.300e+02,
       2.596e+04])

In [134]:
rec = t.reconstruct(area < 2.5243e+04)

In [132]:
rec_masked = ma.masked_equal(rec, 0)

In [133]:
mlab.clf()
idefix.vxl.plot(rec_masked)

Exception occurred in traits notification handler for object: <mayavi.core.lut_manager.LUTManager object at 0x7fe19264b950>, trait: data_range, old value: [0. 1.], new value: [ 1.e+299 -1.e+299]
Traceback (most recent call last):
  File "/home/gentleprotector/miniconda3/envs/letg_internship/lib/python3.12/site-packages/traits/trait_notifiers.py", line 342, in __call__
    self.handler(*args)
  File "/home/gentleprotector/miniconda3/envs/letg_internship/lib/python3.12/site-packages/mayavi/core/lut_manager.py", line 434, in _data_range_changed
    assert value[0] <= value[1]
           ^^^^^^^^^^^^^^^^^^^^
AssertionError


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [129]:
mlab.clf()

In [130]:
masked_before = vxl.mask[vxl.mask == True].size/vxl.size
masked_after = rec_masked.mask[rec_masked.mask == True].size/vxl.size
print(f"Before filtering and reconstructing {masked_before*100:.2f}% of the voxelspace was masked.\nAfter reconstructing {masked_after*100:.2f}% of the space was masked.".format(masked_before, masked_after))

Before filtering and reconstructing 78.94% of the voxelspace was masked.
After reconstructing 79.28% of the space was masked.


**Reconstruction seems to work properly with adapted functionality for SAP**